# App Reviews Grouping

# 1- Initialization

In [1]:
pip install pandas
pip install openpyxl 
pip install demoji

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Zero-Shot Classifier

In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


In [2]:
import pandas as pd                     #python package for data analysis
import numpy as np                      #python package for handling arrays 
import demoji                           #python package for dealing with emojis
df = pd.read_excel("test.xlsx")        #reading the data file and converting into tabular form with pandas
df.head()                              #displaying top 5 records of the data

ID                 Date                AppName  \
0    6  2021-02-06 10:38:31  CapCut - Video Editor   
1    8  2022-01-22 08:56:35  CapCut - Video Editor   
2   61  2021-04-15 01:11:20  CapCut - Video Editor   
3  155  2021-12-06 03:24:56  CapCut - Video Editor   
4  156  2021-08-07 06:00:46  CapCut - Video Editor   

                                              Review  Rating  Unnamed: 5  
0  -\nAfter you exit the app and then open it aga...       4         NaN  
1  - actual motion blur not just a an effect.\n-m...       4         NaN  
2                                      :v it gud lol       5         NaN  
3  。。。。。。。。。。。。。。。。。。。。。\nIm Going To Try To Make...       5         NaN  
4                    ꧁ 𝑏𝑒𝑠𝑡 𝑡𝑤𝑒𝑒𝑛𝑖𝑛𝑔 𝑎𝑝𝑝 ( ˘ ³˘)♥︎ ꧂       5         NaN

In [2]:
# Function to perform all EDA
def perform_eda(df, name=""):
    # Printing basic detail of data like name, size, shape
    print(f"EDA of {str(name)} Data....")
    print(f"Size {df.size}")
    print(f"Columns {df.shape[1]}")
    print(f"Records {df.shape[0]}")
    print("="*50)
    
    # Printing top 5 records of data
    print("First Look of Data....")
    display(df.head())
    print("="*50)
    
    # Getting Numerical and Categorical columns Separately
    cat_cols = df.select_dtypes(np.object).columns
    num_cols = df.select_dtypes(np.number).columns

    # Printing the Numerical columns
    print("Dataset has following Numerical columns...")
    for i, j in enumerate(num_cols):
        print(f" {i+1}) {j}")

    # Printing the Categorical columns
    print("\n\nDataset has following Categorical columns...")
    for i, j in enumerate(cat_cols):
        print(f" {i+1}) {j}")
    
    # Printing info of data like data type, non null values
    print("="*50)
    print("Information of Data....")
    print(df.info())
    print("="*50)
    

# Data Summary

In [3]:
perform_eda(df)

EDA of  Data....
Size 180
Columns 6
Records 30
First Look of Data....


ID                 Date                AppName  \
0    6  2021-02-06 10:38:31  CapCut - Video Editor   
1    8  2022-01-22 08:56:35  CapCut - Video Editor   
2   61  2021-04-15 01:11:20  CapCut - Video Editor   
3  155  2021-12-06 03:24:56  CapCut - Video Editor   
4  156  2021-08-07 06:00:46  CapCut - Video Editor   

                                              Review  Rating  Unnamed: 5  
0  -\nAfter you exit the app and then open it aga...       4         NaN  
1  - actual motion blur not just a an effect.\n-m...       4         NaN  
2                                      :v it gud lol       5         NaN  
3  。。。。。。。。。。。。。。。。。。。。。\nIm Going To Try To Make...       5         NaN  
4                    ꧁ 𝑏𝑒𝑠𝑡 𝑡𝑤𝑒𝑒𝑛𝑖𝑛𝑔 𝑎𝑝𝑝 ( ˘ ³˘)♥︎ ꧂       5         NaN

Dataset has following Numerical columns...
 1) ID
 2) Rating
 3) Unnamed: 5


Dataset has following Categorical columns...
 1) Date
 2) AppName
 3) Review
Information of Data....
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          30 non-null     int64  
 1   Date        30 non-null     object 
 2   AppName     30 non-null     object 
 3   Review      30 non-null     object 
 4   Rating      30 non-null     int64  
 5   Unnamed: 5  0 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.5+ KB
None


In [4]:
labels1 = ['Feature Request', 'Information Giving', 'Problem Discovery']
labels2 = ['Positive', 'Negtive']
filter_keys = ['labels']
#result={}
#result[0]= classifier(sequence_to_classify, candidate_labels)
#result[0]
output=[]
for index, row in df.iterrows():
 d= {}
 sequence = row['Review']
 result1 = classifier(sequence, labels1, multi_class=True)
 result2 = classifier(sequence, labels2, multi_class=True)
 temp = list(map(result1.get, filter_keys))
 temp2 = list(map(result2.get, filter_keys))
 d['ID']= row['ID']
 d['AppName']= row['AppName']
 d['Date']= row['Date']
 d['Rating']= row['Rating']
 d['Review'] = row['Review']
 d['Aspect label'] = temp[0][0]
 d['sentiment label']= temp2[0][0]
 output.append(d) 

 #convert the list of dictionary into pandas DataFrame
new = pd.DataFrame(output)
new


NameError: name 'classifier' is not defined

In [3]:
new=pd.read_csv('results.csv')

In [4]:
tokens_up_lim=250       #setting upper limit of tokens
tokens_low_lim=4        #setting lower limit of tokens

In [5]:
def remove_emoji(text):
    '''
    this function remove the emojis from text so that we
    can find number of token withot emoji
    '''
    emojis=demoji.findall(text)
    for emoji in emojis.keys():
        text=text.replace(emoji,'')
    return text

In [6]:
def reviews_filter(text):
    '''
    this function filter out all the reviews that follows:
    1-no of tokens between 4 and 250
    2- Not only emojis in the review text
    '''
    results=remove_emoji(text)
    no_of_tokens=len(results.split())
    if no_of_tokens>= tokens_low_lim and no_of_tokens <=tokens_up_lim:
        return text

In [7]:
new['Filtered Reviews']=new['Review'].apply(lambda x: reviews_filter(str(x)))

# Results

In [8]:
new['Filtered Reviews']

0     -\nAfter you exit the app and then open it aga...
1     - actual motion blur not just a an effect.\n-m...
2                                         :v it gud lol
3     。。。。。。。。。。。。。。。。。。。。。\nIm Going To Try To Make...
4                       ꧁ 𝑏𝑒𝑠𝑡 𝑡𝑤𝑒𝑒𝑛𝑖𝑛𝑔 𝑎𝑝𝑝 ( ˘ ³˘)♥︎ ꧂
5                           “Add beats” aren’t working.
6     “CapCut needs access to the album for editing ...
7     “Insufficient storage clean up some photos & t...
8     (I’m using iPad Pro if it matters)Ok so I’m no...
9     •No in-app purchases\n•No watermark\n•Extract ...
10    ° Capcut is amazing is so many ways, it’s very...
11    ✝️🔯⛎♌️♑️☪️🕎♈️♍️♒️♓️♎️♉️☯️🕉♊️♏️🆔♐️♋️🛐✡️ search ...
12                     ✨Best editing app in the world✨💅
13    Adddd Graphs please it will make the app 10x b...
14    ADDING GRAPHS AND ACTUAL KEY FRAMES WOULD BE G...
15    adding smooth slowmo & tracking blur might rly...
16    After a while it starts to glitch and lag out ...
17    After all the videos I used to make compar

# Creating Groups

In [125]:
#creating 4 groups as per requirements
reviews_per_group=len(new['Filtered Reviews'])//4+len(new['Filtered Reviews'])%4

In [130]:
groups_df=[]

In [131]:
#this code slice the filtered Reviews into 4 groups
for i,j in  enumerate(range(0, len(new['Filtered Reviews']), reviews_per_group)):
    grp=new['Filtered Reviews'][j:j +reviews_per_group]
    grp=pd.DataFrame(grp)
    grp.columns=[f'Group {i}']
    groups_df.append(grp)

In [169]:
l0=groups_df[0]['Group 0'].tolist()
l0=pd.DataFrame(l0)
l1=groups_df[1]['Group 1'].tolist()
l1=pd.DataFrame(l1)
l2=groups_df[2]['Group 2'].tolist()
l2=pd.DataFrame(l2)
l3=groups_df[3]['Group 3'].tolist()
l3=pd.DataFrame(l3)

In [175]:
groups=pd.concat([l0,l1,l2,l3],axis=1)

In [176]:
groups.columns=['Group 0','Group 1','Group 2','Group 3']

In [178]:
groups

Group 0  \
0  -\nAfter you exit the app and then open it aga...   
1  - actual motion blur not just a an effect.\n-m...   
2                                      :v it gud lol   
3  。。。。。。。。。。。。。。。。。。。。。\nIm Going To Try To Make...   
4                    ꧁ 𝑏𝑒𝑠𝑡 𝑡𝑤𝑒𝑒𝑛𝑖𝑛𝑔 𝑎𝑝𝑝 ( ˘ ³˘)♥︎ ꧂   
5                        “Add beats” aren’t working.   
6  “CapCut needs access to the album for editing ...   
7  “Insufficient storage clean up some photos & t...   
8  (I’m using iPad Pro if it matters)Ok so I’m no...   

                                             Group 1  \
0  •No in-app purchases\n•No watermark\n•Extract ...   
1  ° Capcut is amazing is so many ways, it’s very...   
2  ✝️🔯⛎♌️♑️☪️🕎♈️♍️♒️♓️♎️♉️☯️🕉♊️♏️🆔♐️♋️🛐✡️ search ...   
3                   ✨Best editing app in the world✨💅   
4  Adddd Graphs please it will make the app 10x b...   
5  ADDING GRAPHS AND ACTUAL KEY FRAMES WOULD BE G...   
6  adding smooth slowmo & tracking blur might rly...   
7  After a while it starts to glitch and lag out ...   
8  After all the videos I used to make compared t...   

                                             Group 2  \
0  After being 5 minutes on this app it’s so fun ...   
1  After I cut the video where the beats are and ...   
2  After I edit a video and make another one , th...   
3  after working on an edit for an hour the video...   
4  Ah yes well if it isn’t the app where I can ma...   
5  ah yes, the most used editing app used in a LO...   
6  AHH I have been yawing this app forever and I ...   
7  AHH I have used this app for so long and it wo...   
8  all fonts i try to import say file not support...   

                                             Group 3  
0  All I ask from you is that you make the slomo ...  
1  All I gotta say is it’s better than video star...  
2                all i have to say is i hate you!!!!  
3                                                NaN  
4                                                NaN  
5                                                NaN  
6                                                NaN  
7                                                NaN  
8                                                NaN

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2b12320-9148-49cb-81da-f45d7694199a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>